# Class to  Process Frames from Multiple Video Files  · Basic Usage

In [1]:
cd ../

C:\Users\luisr\Desktop\Repositories\Data Science Projects\Hackaton COR IV - Centro de Operações do RJ\INCUBAÇÃO\Cameras


In [2]:
random_state = 0

#### Function to display train and test classes counting

In [3]:
import pandas as pd

def split_class_count(y_train, y_test):
    display(pd.concat([
        pd.Series(y_train).value_counts().to_frame('Train set'),
        pd.Series(y_test).value_counts().to_frame('Test set')
    ], axis=1))

c:\Users\luisr\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


#### Class to load frames from labeled videos as labeled images

In [18]:
import os, cv2, numpy as np, pandas as pd
from datetime import datetime as dt
from IPython.display import clear_output as co

def timestamp_parser(path):
    file_name = path.split('/')[-1]
    stamp = ' '.join(file_name.split(' ')[1:])
    return dt.strptime(stamp, '%Y-%m-%d %H-%M-%S.mp4')

class VideoImageProcessing:
        
    def __init__(self, dim=1, process=None, fps=3, timestamp_parser=timestamp_parser):
        """  """
        self.dim = dim
        self.process = process
        self.fps = fps
        self.timestamp_parser = timestamp_parser
        
    def apply_to_frames_from_labeled_videos(self, videos, path_key, print_each=None):
        """
        parameters:
            videos - pandas dataframe containing a column with name `path_key`, with paths to a set of videos.
            path_key - name of column containing path to videos in local file system
        """
        i, n  = 0, len(videos)
        results = []
        for idx, video_object in videos.iterrows():
            frames_results = self.apply_to_frames(video_object, path_key)
            results += frames_results
            i += 1
            if print_each is not None and i % print_each == 0:
                co(True); print(f'LABELED VIDEOS PROCESSING · PROGRESS: {i}/{n}')
        return results

    def apply_to_frames(self, video_object, path_key):
        path = video_object[path_key]
        timestamp = None
        if self.timestamp_parser is not None:
            timestamp = self.timestamp_parser(path)
            offset = pd.offsets.Second() / self.fps
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"CANNOT OPEN VIDEO CAPTURE · PATH: {path}")
            return []
        frames_results = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break # stream finished
            if self.dim == 1: # 1D flat frame
                frame = np.reshape(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), -1)
            if self.dim == 2: # 2D gray scale frame
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if self.timestamp_parser is not None:
                timestamp += offset # frame time stamp update
            result = self.process(frame, metadata=video_object.to_dict(), timestamp=timestamp)
            frames_results.append(result)
        cap.release(); cv2.destroyAllWindows()
        return frames_results

#### Class to write videos

In [45]:
import os, cv2

class Video:

    def __init__(self, codec:str='mp4v', fps:int=3, shape:tuple=(854, 480), overwrite=False):
        self.codec = codec; self.fps = fps; self.shape = shape
        self.overwrite = overwrite

    def writer(self, path):
        if not self.overwrite and os.path.exists(path):
            print(f'ANNOTATE VIDEO TIMESTAMP FAILED. FILE ALREADY EXISTS · FILE-PATH: {path}')
            return False
        return cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*self.codec), self.fps, self.shape)

#### List nested sub-directories from given folder

In [20]:
import os, cv2, pandas as pd
from datetime import datetime as dt
from IPython.display import clear_output as co

def inner_subdirs(folder, ext='.mp4'):
    if folder.endswith('/'): folder = folder[:-1]
    folder_depth = len(folder.split('/'))
    in_subdirs = []
    for path, subdirs, files in os.walk(folder):
        if os.path.isdir(path) and any([file.endswith(ext) for file in files]):
            subpath = '/'.join(path.split('\\')[1:])
            in_subdirs.append(subpath)
    return in_subdirs

---
## Reload labeled flood videos dataset

In [21]:
import pandas as pd

videos = pd.read_csv('Dados/Rotulos/1475_2023-02-07.csv')

# preprocessing - blob/file name fix · replace `:` for `-`
videos['blob_name'] = [blob_name.replace(':', '-') for blob_name in videos['blob_name']]

# preprocessing - datetime conversion
videos['timestamp'] = pd.to_datetime(videos['timestamp'])
videos = videos.set_index('timestamp', drop=True).sort_index()

# preprocessing - drop videos larger than `video_max_bytes`
video_max_bytes = 5e6 # 5 Mb
original_shape = videos.shape
videos = videos[videos['blob_size'] < video_max_bytes]

display(videos[['blob_name', 'tag']].head())
print(f'Shape: {original_shape}')
print(f'\n - Shape after processing: {videos.shape}')

,blob_name,tag
timestamp,,
2023-02-07 19:25:00,polygons/flood-unlabeled/1/1475/CODE1475 2023-...,alagamento
2023-02-07 19:30:00,polygons/flood-unlabeled/1/1475/CODE1475 2023-...,alagamento
2023-02-07 19:35:00,polygons/flood-unlabeled/1/1475/CODE1475 2023-...,alagamento
2023-02-07 19:40:00,polygons/flood-unlabeled/1/1475/CODE1475 2023-...,alagamento
2023-02-07 19:50:00,polygons/flood-unlabeled/1/1475/CODE1475 2023-...,alagamento


Shape: (165, 8)

 - Shape after processing: (163, 8)


---
## Video Data Preprocessing

#### Binarize target variable categories

In [22]:
replace_tags = {
    'acúmulo': ['lâmina', 'bolsão', 'alagamento'],
    'normalidade': ['poça', 'normalidade'],
}

y_true = []
for tag in videos['tag']:
    for key, values in replace_tags.items():
        if tag in values: y_true.append(key)

y_true = pd.Series(y_true, index=videos.index)

videos['label'] = y_true

display(videos['label'].value_counts().to_frame('Video Samples'))

,Video Samples
label,
normalidade,128
acúmulo,35


---
## Video Data Sampling

#### Optional · Train and test split of video samples

In [23]:
from sklearn.model_selection import train_test_split

t_size = 0.04
e_size = 0.04

# X, Y = x_res, y_res  # pre-sampled
X, Y = videos, videos['label']  # complete set of videos

xt, xe, yt, ye = train_test_split(
    X, Y, test_size=e_size, train_size=t_size,
    random_state=random_state, shuffle=True, stratify=Y
)

# complete x and y sequences
yy = pd.concat([yt, ye], axis=0)
xx = pd.concat([xt, xe], axis=0)

split_class_count(yt, ye)
display(yy.value_counts().to_frame('Complete sample set'))

,Train set,Test set
label,,
normalidade,5,5
acúmulo,1,2


,Complete sample set
label,
normalidade,10
acúmulo,3


---
## Load Flood Classifier Model

#### Load flood image classifier

In [24]:
import pickle, pandas as pd

filename = 'code1475.sav'
model = pickle.load(open(filename, 'rb'))

#### Image processing function

In [25]:
# Get image classification prediction and metadata

def classify_image(frame, metadata, timestamp=None):
    prediction = model.predict([frame])[0]
    return pd.Series({
        'prediction': prediction,
        'timestamp': timestamp,
        **metadata
    })

---
## Classifiy images from selected labeled videos

#### Videos frames pipeline · Frame image classification

In [78]:
folder = 'Dados/flood-video-collection-stamped'
process = classify_image

# XX, YY = xx.copy(), yy.copy()  # pre-sampled from loaded videos dataset
XX, YY = videos.copy(), videos['label'].copy()  # complete loaded videos dataset

XX['path'] = [f'{folder}/{blob_name}' for blob_name in XX['blob_name']]

pipeline = VideoImageProcessing(dim=1, process=process)
results= pipeline.apply_to_frames_from_labeled_videos(videos=XX.reset_index(), path_key='path', print_each=1)

print(f'\nVideos Selected: {len(XX)}')
print(f'Frames Loaded: {len(results)}')
print()

LABELED VIDEOS PROCESSING · PROGRESS: 163/163

Videos Selected: 163
Frames Loaded: 5419



#### Evaluate result

In [76]:
frames = pd.DataFrame(results)

#### Classification Simulation Report

from sklearn.metrics import classification_report

# Get frames labels and predictions
y_true = frames['label']
y_pred = frames['prediction']

print(classification_report(y_true, y_pred))
print(f'\n * Accuracy: {round((y_true == y_pred).mean(), 2)}')
print(f'\n * Mislabeled data points: {(y_true != y_pred).sum()} / {len(y_true)}')

              precision    recall  f1-score   support

     acúmulo       0.00      0.00      0.00       105
 normalidade       0.77      1.00      0.87       347

    accuracy                           0.77       452
   macro avg       0.38      0.50      0.43       452
weighted avg       0.59      0.77      0.67       452


 * Accuracy: 0.77

 * Mislabeled data points: 105 / 452


c:\Users\luisr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luisr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\luisr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


---
## Concatenate labeled videos from single camera

#### Custom functions to process videos' frames individually

In [134]:
def add_text_to_video(frame, text):
    cv2.putText(frame, text, org=(542, 27), fontFace=cv2.FONT_HERSHEY_COMPLEX,
    fontScale=0.67, color=(0, 0, 0), thickness=2, lineType=cv2.LINE_8)
    cv2.putText(frame, text, org=(540, 25), fontFace=cv2.FONT_HERSHEY_COMPLEX,
    fontScale=0.67, color=(40, 230, 230), thickness=2, lineType=cv2.LINE_8)
    return frame

def history_write(frame, metadata, timestamp):
    frame_1d = np.reshape(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), -1) # 1D from gray 2D image (reduces columns 3 times)
    timestamp_str =  timestamp.strftime('OCTA %d/%m/%Y %H:%M:%S')
    label = metadata['label']
    prediction = model.predict([frame_1d])[0]
    frame = add_text_to_video(frame, timestamp_str)  # put timestamp text
    text_label = f'Rotulo: {label.upper().replace("Ú", "U")}'
    text_prediction = f'Previsao (IA): {prediction.upper().replace("Ú", "U")}'
    cv2.putText(
        frame, text_label, org=(20, 425), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.67, color=(40, 230, 230), thickness=2
    )
    cv2.putText(
        frame, text_prediction, org=(20, 460), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.67, color=(40, 230, 230), thickness=2
    )
    history_writer.write(frame)
    return {
#         'frame': frame,
        'write': 'success',
        'prediction': prediction,
        'timestamp': timestamp,
        **metadata,
    }

#### Videos frames pipeline · Concatenate multiple videos

In [137]:
folder = 'Dados/flood-video-collection'
full_history_video_path = 'Dados/Simulação/history-labeled-code1475.mp4'
process = history_write

# XX, YY = xx.copy(), yy.copy()  # pre-sampled from loaded videos dataset
XX, YY = videos.copy(), videos['label'].copy()  # complete loaded videos dataset

XX['path'] = [f'{folder}/{blob_name}' for blob_name in XX['blob_name']]

# OPEN VIDEO FILE WRITER
video = Video(overwrite=True, fps=15)
history_writer = video.writer(path=full_history_video_path)

# RUN FRAME PIPELINE
pipeline = VideoImageProcessing(dim=3, process=process)
results = pipeline.apply_to_frames_from_labeled_videos(videos=XX.sort_index().reset_index(), path_key='path', print_each=1)

# CLOSE/RELEASE VIDEO FILE WRITER
history_writer.release(); cv2.destroyAllWindows()

print(f'\nVideos Selected: {len(XX)}')
print(f'Frames Loaded: {len(results)}')
print()

LABELED VIDEOS PROCESSING · PROGRESS: 163/163

Videos Selected: 163
Frames Loaded: 5419



#### Evaluate result

In [136]:
frames_concat = pd.DataFrame(results)

#### Classification Simulation Report

from sklearn.metrics import classification_report

# Get frames labels and predictions
y_true = frames_concat['label']
y_pred = frames_concat['prediction']

display(frames_concat['write'].value_counts().to_frame('Operation Success'))
print()

print(classification_report(y_true, y_pred))
print(f'\n * Accuracy: {round((y_true == y_pred).mean(), 2)}')
print(f'\n * Mislabeled data points: {(y_true != y_pred).sum()} / {len(y_true)}')

,Operation Success
write,
success,5419



              precision    recall  f1-score   support

     acúmulo       0.94      0.94      0.94       978
 normalidade       0.99      0.99      0.99      4441

    accuracy                           0.98      5419
   macro avg       0.96      0.97      0.96      5419
weighted avg       0.98      0.98      0.98      5419


 * Accuracy: 0.98

 * Mislabeled data points: 115 / 5419
